# Neural Network with 2D Data
- Neural Network with 5 layers
- 2D data f(x,y) -> R

## 1.Import Required Libraries

In [1]:
!pip3 install visdom

     |████████████████████████████████| 676 kB 5.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=6021e4b37158102b33b5395166cfa0d58a363fad3ebf8b814cd762e18b565743
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=20dec346d964692e01e8102d7f4eb0f0e1da1e37cefac69e9362dcc23a2be5ae
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom

In [3]:
# 코렙에서 visdom 사용 방법

# !npm install -g npm to update!
! npm install -g localtunnel
get_ipython().system_raw('/usr/local/bin/python -m pip install visdom')
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

import time
time.sleep(5)
! cat url.txt

import visdom
time.sleep(5)
vis = visdom.Visdom(port='6006')
print(vis)
time.sleep(3)
vis.text('testing')
! cat visdomlog.txt
# 아래 출력 메세지의 Your url is 부분의 url 클릭함
# 나타난 웹 페이지의 'click to countinue' 버튼 클릭함

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.817s
your url is: https://witty-cobra-36.loca.lt


Setting up a new session...


/usr/local/lib/python3.7/dist-packages/visdom/server.py:39: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  ioloop.install()  # Needs to happen before any tornado imports!
INFO:root:Application Started
INFO:tornado.access:200 POST /env/main (127.0.0.1) 0.96ms
INFO:tornado.access:101 GET /vis_socket (127.0.0.1) 0.40ms
INFO:root:Opened visdom socket from ip: 127.0.0.1
INFO:tornado.access:200 POST /events (127.0.0.1) 0.70ms


## 2. Data Generation

In [4]:
num_data = 1000
num_epoch = 10000

x = init.uniform(torch.Tensor(num_data, 1), -10, 10)
y = init.uniform(torch.Tensor(num_data, 1), -10, 10)
z = x**2 + y**2

x_noise = init.normal(torch.FloatTensor(num_data, 1), std=0.5)
y_noise = init.normal(torch.FloatTensor(num_data, 1), std=0.5)
z_noise = x_noise**2 + y_noise**2

data_noise = torch.cat([x, y, z_noise], 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  if __name__ == '__main__':


In [6]:
# visualize data with visdom

win_1=vis.scatter(
        X=data_noise,
        opts=dict(
            markersize=5,
            markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=[51,153,255]*np.ones(shape=[num_data,3]))
            )
        )


## 3. Model & Optimizer

In [7]:
# fully connected model with 5 hidden layer

model = nn.Sequential(
        nn.Linear(2,20),
        nn.ReLU(),
        nn.Linear(20,10),
        nn.ReLU(),
        nn.Linear(10,5),
        nn.ReLU(),
        nn.Linear(5,5),
        nn.ReLU(),
        nn.Linear(5,1),
    ).cuda()

loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(),lr=0.001)

## 4. Train 

In [10]:
input_data = torch.cat([x,y],1).cuda()
label = z_noise.cuda()
loss_arr =[]

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(input_data))
    loss = loss_func(output,Variable(label))
    loss.backward()
    optimizer.step()
    
    loss_arr.append(loss.cpu().data.numpy())
    
    if i % 100 == 0 and i <1000:
        print(loss)
        data = torch.cat([input_data.cpu(),output.cpu().data],1)

        win_2 =vis.scatter(
                X=data,
                opts=dict(
                markersize=5,
                markercolor=np.ndarray(shape=[num_data,3],
                                       dtype=float,
                                       buffer=128*np.ones(shape=[num_data,3]))
            )
    )

tensor(0.6549, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.4655, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3915, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3730, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3662, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3623, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3592, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3566, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3545, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(0.3528, device='cuda:0', grad_fn=<L1LossBackward0>)


In [11]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[-0.6762, -0.5715],
        [ 0.5086, -0.3387],
        [ 0.0054,  0.1475],
        [ 0.1425,  0.0778],
        [-0.5137, -0.7001],
        [-0.4910,  0.6736],
        [ 0.6617, -0.6077],
        [-0.3801, -0.5168],
        [-0.0471,  0.0390],
        [ 0.3154,  0.2916],
        [-0.3585, -0.2049],
        [ 0.2859, -0.1678],
        [-0.5311, -0.0046],
        [ 0.4625, -0.0217],
        [-0.6461, -0.1778],
        [-0.3547, -0.1671],
        [-0.0436,  0.4547],
        [-0.2971,  0.6078],
        [-0.3225, -0.2553],
        [-0.6681,  0.5557]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2215,  0.5034, -0.1228,  0.1763,  0.3108, -0.3234, -0.7038,  0.1927,
        -0.1368,  0.1249, -0.2213, -0.6031,  0.2521, -0.3657,  0.0054, -0.2190,
         0.4882,  0.1685, -0.5407,  0.3222], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([[ 0.1078,  0.1757, -0.0298, -0.1710,  0.2118, -0.0866,  0.0095,  0.0182,
   

## 5. Visualize Trained Output

In [13]:
data = torch.cat([input_data.cpu(),output.cpu().data],1)

win_2 = vis.scatter(
    X = data,
    opts=dict(        
        markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=128*np.ones(shape=[num_data,3])),
        markersize=5,
    ),
)


## 6. Visualize Loss Graph

In [14]:
x = np.reshape([i for i in range(num_epoch)],newshape=[num_epoch,1])
loss_data = np.reshape(loss_arr,newshape=[num_epoch,1])

win3 = vis.line(
    X = x.reshape(x.shape[0]),
    Y = loss_data.reshape(loss_data.shape[0]),
)